# HW 5

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import time

In [ ]:
# In a terminal install the cvxpy library -> "pip install cvxpy" 
# You may need to add the --user flag if you don't have admin privileges
import cvxpy as cp
import IPython.display as ipd


## 1. Optimization

In class you have learned how to form optimization problems via formulations of minimization, constraints, and penalties. 
We will now use the library CVXPY to solve the problem on your quiz as well as create a more elaborate optimization problem.

### 1.0 CVXPY Tutorial

The code block below shows you the basics of using the optimization library. <br>
It optimizes the following problem: <br>
<br>
$argmin_{x} ||y-Ax||^{2}_{2}$ <br>
subject to $x \geq 0$, $x \leq 1$

In [ ]:
# Problem data.
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n) # creates a random matrix of size m x n
y =  np.random.randn(m) # creates a random vector of size m

# Construct the problem.
x = cp.Variable(n) # creates a variable vector of size n to be optimized
objective = cp.Minimize(cp.sum_squares(A @ x - y)) # '@' is the matrix multiplication operator
constraints = [0 <= x, x <= 1] # creates a list of constraints
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)

### 1.1 Solve Quiz 13 Problem

You are tasked with programming the motor of a robotic arm to perform various motions.
Your task is to match a theoretical motion, y, with a motion, x, that closely matches it under the
physical constraints of the motor. The constraint is that neighboring elements in x cannot have an
absolute difference greater than 5 units (the motor cannot have a change in position greater than 5
over 1 discrete unit of time). 

Assume the sampling frequency here is 5 Hz.

In [ ]:
# The desired motion is hardcoded in the variable 'y' and plotted below
t = np.arange(0,20,1)/5
y = np.array([0,6,8,9,10,2,2,2,8,8,8,2,1,1,1,9,1,9,1,0,])

plt.figure(figsize=(10,5))
plt.plot(t,y, '-o')
plt.xlabel('Time (s)')
plt.ylabel('Position')
plt.title('Desired Motion')
plt.show()

In [ ]:
################# Edit the code below to solve for X #################
# You may want to create the matrix D here
# D = ...

x = cp.Variable(n) # creates a variable vector of size n to be optimized
objective = cp.Minimize(...) # Fill in the objective function
constraints = [...] # Fill in the constraints
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)

# The optimal motion is stored in the variable 'x' and plotted below
plt.figure(figsize=(10,5))
plt.plot(t, y, '-o', label='Desired Motion')
plt.plot(t, x.value, '-o', label='Physically Realizable Motion')
plt.xlabel('Time')
plt.ylabel('Position')
plt.title('Physically Realizable Motion vs. Desired Motion')
plt.legend()
plt.show()### 1.1 Solve Quiz 13 Problem

print(f"Error between desired and realized motion: {np.sum(np.abs(y-x.value))}")

### 1.2 Adding Penalties

Now assume that the robotic arm degrades over time when performing motion at a certain frequency or higher. <br>
We would like to add a penalty term to the Minimization to penalize frequecies that are equal to or higher than 1 Hz. <br>
Formally, put a penalty on the square of the magnitude of frequencies equal to or higher than 1 Hz. <br>
<br>
Hint: The function in the cvpxy library 'cp.abs()' may be useful. <br>
Suggestion: Writing out the optimization problem on paper first may be helpful.

In [ ]:
################# Edit the code below to solve for X #################

# You may want to create the matrix D here
# D = ...

# As well as the DFT matrix
# DFT_matrix = ...

x = cp.Variable(n) # creates a variable vector of size n to be optimized
alpha = 1e-1
objective = cp.Minimize(cp.sum_squares(...) + alpha*cp.sum_squares(...)) # Fill in the objective function as well as the penalty term
constraints = [...] # Fill in the constraints
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)

# The optimal motion is stored in the variable 'x' and plotted below
plt.figure(figsize=(10,5))
plt.plot(t, y, '-o', label='Desired Motion')
plt.plot(t, x.value, '-o', label='Physically Realizable Motion')
plt.xlabel('Time')
plt.ylabel('Position')
plt.title('Physically Realizable Motion vs. Desired Motion')
plt.show()### 1.1 Solve Quiz 13 Problem

print(f"Error between desired and realized motion: {np.sum(np.abs(y-x.value))}")

Question 1: Why is the error between the desired and realized motion larger in Section 1.3 than Section 1.2?

Answer: 

Question 2: Why does the solution in Section 1.3 differ greatly from the solution in Section 1.2 around the time 3-3.5 seconds?

Answer: 

## 2. FFT

In HW 4 you were asked to code the DFT and its properties from scratch. Since you know now how it works, we can use Python libraries that have optimized functions for computing the DFT to compare with out Naive implementation.

### DFT vs. FFT Speed Comparison

In this question, we want to measure how fast the FFT is compared to a naive implementation of the DFT. Some helper functions have been added for you. <br>
In order to see the difference, plot the time it takes to compute the DFT vs. FFT for different length signals of size (N). Make sure your plot spans from values of N from 2 to 16384.

In [ ]:
def get_dft_matrix(x_signal):
    N = len(x_signal)
    dftmtx = np.fft.fft(np.eye(N))
    return dftmtx

def dft(x_signal):
    dftmtx = get_dft_matrix(x_signal)
    return np.matmul(dftmtx, x_signal)

def generate_random_signal(N):
    return np.random.randn(N)

In [ ]:
################# Write you code and make your plot below in this cell #################
# Your generated plot should have two curves, one for DFT and one for FFT
# The x-axis should be the length of the signal, and the y-axis should be the time it takes to compute the DFT/FFT
# Hint: use the time module and the time.time() function to measure the time it takes for computation
# Hint: use the np.fft.fft function to compute the FFT
# Hint: Use the functions defined above to compute the DFT


Question 2: What do you observe about the computational complexity of the naive DFT and FFT algorithms?

Answer: 